In [93]:
import sys

sys.api_version
!{sys.executable} -m pip install -qUr requirements.txt

In [94]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_openai import OpenAIEmbeddings
from package import OPENAI_API_KEY, set_environment_openai_api_key
from package.utilities import get_file_contents
set_environment_openai_api_key()

chat = ChatOpenAI(model='gpt-4o', openai_api_key=OPENAI_API_KEY)
orchestrator_prompt = get_file_contents('orchestrator-prompt.txt')
entities = get_file_contents('entities.txt').splitlines()

In [95]:
aiMessage: AIMessage = chat.invoke([
    SystemMessage(content=orchestrator_prompt),
    HumanMessage(content="The manager of chat-bot's project has been changed, now it's Ivan Petrov")
])
candidates = [candidate.lstrip('0123456789.- ') for candidate in aiMessage.content.splitlines()]
print(candidates)

['Manager', 'Chat-bot', 'Project', 'Ivan Petrov']


In [96]:
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', headers={'x-api-key': OPENAI_API_KEY})
retriever = FAISS.from_texts(entities, embeddings).as_retriever()
for candidate in candidates:
    print(candidate, ' -> ', retriever.invoke(candidate)[0].page_content)

Manager  ->  employee
Chat-bot  ->  bot program
Project  ->  project
Ivan Petrov  ->  director


In [99]:
from langchain_community.vectorstores import Neo4jVector

title = 'title'

embeddings2 = OpenAIEmbeddings(model='text-embedding-ada-002', headers={'x-api-key': OPENAI_API_KEY})
vector_index = Neo4jVector.from_existing_graph(
    embeddings2,
    url='bolt://localhost:7687',
    username='neo4j',
    password='neo4j1234',
    index_name='node_text_index_title',
    node_label='Term',
    text_node_properties=[title],
    embedding_node_property='embedding',
)
for candidate in candidates:
    response = vector_index.similarity_search(
        f"Which `Term` is nearest for `{candidate}`?"
    )
    print(candidate, ' -> ', response[0].page_content.strip().removeprefix(f"{title}: "))

ValueError: Index with name vector already exists.The provided embedding function and vector index dimensions do not match.
Embedding function dimension: 3072
Vector index dimension: 1536